# Azure AI Agent service - Function calling

<img src="agent.jpg" width=800>
<img src="foundry.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import json
import os
import requests
import sys
import time
import yfinance as yf

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Any, Callable, Set, Dict, List, Optional

from user_functions import user_functions, azuremaps_weather

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## 1. Agent definition - Weather forecasts

### Project client

In [5]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

In [6]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

### Testing the Azure Maps Weather function

We will the Azure Maps service to provide weather information.

In [7]:
weather = azuremaps_weather("Seattle")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-04-30T01:32:00-07:00',
    'phrase': 'Cloudy',
    'iconCode': 7,
    'hasPrecipitation': False,
    'isDayTime': False,
    'temperature': {'value': 9.3, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 9.5, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 9.5, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 82,
    'dewPoint': {'value': 6.4, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 135.0, 'localizedDescription': 'SE'},
     'speed': {'value': 1.8, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 5.6, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 0,
    'uvIndexPhrase': 'Low',
    'visibility': {'value': 9.7, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 100,
    'ceiling': {'value': 3048.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1024.4, 'unit': 'mb', 'unitType': 14},
    'pressureTendency

### Agent

In [8]:
model = "gpt-4o-mini"
name = "gpt4o-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

# user functions to use
user_functions: Set[Callable[..., Any]] = {
    azuremaps_weather,
}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# We need to define this option to the project (not in the documentation ?)
project_client.agents.enable_auto_function_calls(toolset=toolset)

# Agent creation
agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    toolset=toolset,
)

print(f"Created agent with ID = {agent.id}")

Created agent with ID = asst_rxjxPAouxWUmGwXTDcyM6kT7


### Tests

In [9]:
prompt = "Hello, generate a full report for the weather in Paris today"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_eawWAheM6cmVucETN2srlYFL
Created message ID = msg_1yHP1WPGSdUOhsWJjgwvHJAF


In [10]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_6WFuYielwDK6jXwP1jV84hQF


In [11]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

Start   : 2025-04-30 08:46:28+0000
End     : 2025-04-30 08:46:31+0000
Elapsed : 0:00:03  (3.00 seconds)


In [12]:
run.usage

{'prompt_tokens': 1496, 'completion_tokens': 392, 'total_tokens': 1888, 'prompt_token_details': {'cached_tokens': 0}}

In [13]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


### Weather Report for Paris, France (April 30, 2025)

#### General Conditions
- **Date and Time**: April 30, 2025, 10:26 AM (Local Time)
- **Weather**: Sunny
- **Icon**: ☀️ (Clear/Sunny)

#### Temperature
- **Current Temperature**: 20.1°C
- **Feels Like**: 23.0°C
- **Real Feel Temperature Shade**: 18.7°C

#### Humidity and Dew Point
- **Relative Humidity**: 48%
- **Dew Point**: 8.7°C

#### Wind
- **Direction**: Northeast (NE) at 45°
- **Speed**: 9.6 km/h
- **Wind Gusts**: 15.6 km/h

#### Visibility and Air Pressure
- **Visibility**: 24.1 km
- **Pressure**: 1022 mb (Steady)

#### Cloud Cover and Ceiling
- **Cloud Cover**: 0% (Clear)
- **Ceiling**: 12192 meters

#### UV Index
- **UV Index**: 5 (Moderate)

#### Precipitation
- **Precipitation Summary**: No precipitation recorded in the past 24 hours.

#### Temperature Summary (Last 24 Hours)
- **Minimum Temperature**: 12.3°C
- **Maximum Temperature**: 25.4°C

#### Additional Conditions
- **Apparent Temperature**: 19.4°C
- **Wind Chill T

### Other examples

In [14]:
prompt = "Hello, generate a full report for the weather in London today. Print the results in English and French."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_YMeiEvtymOk9mm0FUDtjJmtn
Created message ID = msg_0270gf7nD2B19j1gQwf9zFPl


In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


### Weather Report for London Today

**English Version:**

- **Date:** April 30, 2025
- **Condition:** Sunny
- **Temperature:** 16.6°C
- **Real Feel Temperature:** 20.2°C
- **Humidity:** 67%
- **Dew Point:** 10.4°C
- **Wind:** ENE at 7.7 km/h
- **Wind Gusts:** 17.5 km/h
- **UV Index:** Moderate (4)
- **Visibility:** 16.1 km
- **Cloud Cover:** 0%
- **Pressure:** 1023 mb (Falling)
- **Apparent Temperature:** 18.3°C
- **Wind Chill Temperature:** 16.7°C
- **Wet Bulb Temperature:** 13.2°C
- **Precipitation:** None in the past 24 hours
- **Temperature Summary:** Maximum of 21.6°C and Minimum of 9.0°C in the past 24 hours

---

**Version Française :**

- **Date :** 30 avril 2025
- **Condition :** Ensoleillé
- **Température :** 16,6°C
- **Température ressentie :** 20,2°C
- **Humidité :** 67%
- **Point de rosée :** 10,4°C
- **Vent :** ENE à 7,7 km/h
- **Rafales de vent :** 17,5 km/h
- **Indice UV :** Modéré (4)
- **Visibilité :** 16,1 km
- **Couverture nuageuse :** 0%
- **Pression :** 1023 mb 

In [16]:
prompt = "Hello, print the UV informations for London today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_OUImF2PjeYAPHnneEgCas1Yy
Created message ID = msg_hCkBzMqbl2Ocw50BtJBgo6ih


In [17]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The UV information for London today is as follows:

- **Date**: April 30, 2025
- **UV Index**: 4
- **UV Index Phrase**: Moderate

The weather is currently sunny with a temperature of 16.6°C.


In [18]:
prompt = "Hello, What is the temperature in Paris? Display the time of the forecast"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_TE9lhmahtszL6u8PF5oXk22b
Created message ID = msg_SqhIoVDvLt5u5i7LVYcrKa3r


In [19]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The current temperature in Paris is **20.1°C**. The forecast was last updated on **April 30, 2025, at 10:26 AM (local time)**. The weather is currently sunny with a moderate UV index.


In [20]:
prompt = "Hello, What is the temperature and UV information for Marseille? Display the time of the forecast for the UTC zone"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_HddVdwxA4cCB2wWXLDWSlx6G
Created message ID = msg_XNkHJMh27knG0SDt6THyLIVc


In [21]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The current weather conditions for Marseille are as follows:

- **Temperature**: 20.0°C
- **UV Index**: 5 (Moderate)

The forecast time is **2025-04-30 at 08:36 UTC**.


In [22]:
prompt = "Generate a weather report for Paris. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Y7JsqA8hsFuZquGx1r1aMpO2
Created message ID = msg_eS2uTTXed8mtSbb83V9fX893


In [23]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


🌞 **Bonjour, Paris Enthusiasts!** 🌞

Get ready to bask in the glow of the City of Lights because today is absolutely **sunny** (iconic, right? ☀️)! 

🍃 **Current Vibes:**
- **Temperature**: A delightful **20.6°C** – perfect for strolling along the Seine or sipping a café au lait! ☕️
- **Real Feel**: Thanks to the sunshine, it feels like a warm **23.5°C**! So, don't forget to slap on some sunscreen. 🧴
- **Humidity**: A comfy **46%** means it’s not too sticky.
- **Wind**: A gentle whisper from the **northeast** at **9.6 km/h** – just enough to rustle your stylish beret! 🎩

🌤️ **Today's Highlights**:
- **UV Index**: Moderate (5). So while you're soaking up the sun, keep it safe! 😎
- **Visibility**: A fantastic **24.1 km**, ensuring you can see every beautiful moment unfold around you.
- **Precipitation**: Zero! 🌈 You won't need that umbrella unless you're planning to dodge the occasional pigeon.

💨 **A Glimpse Ahead**: The temperature has been a steady companion, with the past 24 hours s

### Post processing

In [24]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)

Deleting agent asst_rxjxPAouxWUmGwXTDcyM6kT7
